In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('team_roster_data_untouched.csv')

In [20]:
print(df.columns)
df.dropna(subset=["firstName", "lastName", "homeLatitude", "homeLongitude", "homeCity", "homeState", "homeCountry"], inplace=True)
df.reset_index(drop=True, inplace=True)

Index(['id', 'firstName', 'lastName', 'team', 'weight', 'height', 'jersey',
       'year', 'position', 'homeCity', 'homeState', 'homeCountry',
       'homeLatitude', 'homeLongitude', 'homeCountyFIPS', 'recruitIds'],
      dtype='object')


In [21]:
p = {}
current_ids = set[str]()

for i in range(len(df)):

    # Get data
    team = df.loc[i, 'team']
    first, last = df.loc[i, 'firstName'], df.loc[i, 'lastName']
    name = f"{first} {last}"
    ID = df.loc[i, 'id']
    year = df.loc[i, 'year']
    homeTown = df.loc[i, 'homeCity']
    homeState = df.loc[i, 'homeState']
    homeCountry = df.loc[i, 'homeCountry']

    # Check if there is a new name
    if ID not in current_ids:
        current_ids.add(ID)
        p[ID] = {
            "name": name,
            "teams": [team],
            "year": [year],
            "homeLat": [df.loc[i, 'homeLatitude']],
            "homeLong": [df.loc[i, 'homeLongitude']],
            "homeTown": [homeTown],
            "homeState": [homeState],
            "homeCountry": [homeCountry]
        }
    else:
        p[ID]["teams"].append(team)
        p[ID]["year"].append(year)

    

In [22]:
edges = {
    "name" : [],
    "new_recruit" : [],
    "team" : [],
    "year" : [],
    "prev_team" : [],
    "homeLat" : [],
    "homeLong" : [],
    "homeTown" : [],
    "homeState" : [],
    "homeCountry" : []
}

# NOTE 
""" I want to make a df containting edges

id | name | new_recruit(bool) | team | year | prev_team | homeLat | homeLong |homeTown

to make it easier to create the edges matrix
"""

for ID, data in p.items():

    # Add the recruiting time
    edges["name"].append(data['name'])
    edges["new_recruit"].append(True)
    edges["team"].append(data['teams'][0])
    edges["year"].append(data['year'][0])
    edges["prev_team"].append(0)
    edges["homeLat"].append(data['homeLat'])
    edges["homeLong"].append(data['homeLong'])
    edges["homeTown"].append(data['homeTown'])
    edges["homeState"].append(data['homeState'])
    edges["homeCountry"].append(data['homeCountry'])

    # Get all additional years
    for i in range(1, len(data['teams'])):
        if data['teams'][i] != data['teams'][i-1]:
            # Add the new team
            edges["name"].append(data['name'])
            edges["new_recruit"].append(False)
            edges["team"].append(data['teams'][i])
            edges["year"].append(data['year'][i])
            edges["prev_team"].append(data['teams'][i-1])
            edges["homeLat"].append(0)
            edges["homeLong"].append(0)
            edges["homeTown"].append(data['homeTown'])
            edges["homeState"].append(data['homeState'])
            edges["homeCountry"].append(data['homeCountry'])
            
# Save the edges df
edges_df = pd.DataFrame(edges)
edges_df.to_csv('edges.csv', index=False)